https://vgpena.github.io/classifying-tweets-with-keras-and-tensorflow/

En el anterior enlace, tenéis un ejemplo sobre cómo, a partir de tweets con un label específico (un sentimiento, positivo o negativo): 

1. Genera un conjunto de entrenamiento. El conjunto de entrenamiento es formado a partir de tweets completos pasados a un array con un tamaño específico.
2. Ese array (X_train de tamaño N) tiene un label que representa el sentimiento (y_train)
3. Como todas las frases tienen un tamaño N, la entrada de la red neuronal será de tamaño N y la salida de la red será de tamaño 2 usando activación softmax(porque hay dos clases).

Se pide: 

- Realizar un clasificador de reviews para el dataset de IMDB de la carpeta data_exercise/

**Cuando usa la importación "keras.x", reemplázalo por "tensorflow.keras.x"**

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from tensorflow import keras
import tensorflow.keras.preprocessing.text as kpt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation

In [5]:
training = pd.read_csv("data_exercise/IMDB Dataset.csv")
training.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
training["sentiment"].value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

### TRAIN, TEST

In [6]:
X_train = training["review"]
y_train = training["sentiment"]

In [9]:
y_train = y_train.apply(lambda x: 1 if x == "positive" else 0)

In [10]:
y_train = np.asarray(y_train)

In [11]:
max_words = 3000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

In [13]:
dictionary = tokenizer.word_index

with open('dictionary.json', 'w') as dictionary_file:
    json.dump(dictionary, dictionary_file)

In [14]:
def convert_text_to_index_array(text):
    return [dictionary[word] for word in kpt.text_to_word_sequence(text)]

In [15]:
allWordIndices = []

for text in X_train:
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

In [17]:
allWordIndices = np.asarray(allWordIndices)

train_x = tokenizer.sequences_to_matrix(allWordIndices, mode='binary')

train_y = keras.utils.to_categorical(y_train, 2)

C:\Users\xyang\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


## MODEL

In [18]:
model = Sequential()
model.add(Dense(512, input_shape=(max_words,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
  optimizer='adam',
  metrics= ["CategoricalAccuracy"])

In [19]:
hist_model = model.fit(train_x, train_y,
  batch_size=32,
  epochs=5,
  verbose=1,
  validation_split=0.1,
  shuffle=True)

Epoch 1/5
1407/1407 [==============================] - 17s 11ms/step - loss: 0.3317 - categorical_accuracy: 0.8566 - val_loss: 0.2732 - val_categorical_accuracy: 0.8826
Epoch 2/5
1407/1407 [==============================] - 13s 9ms/step - loss: 0.2490 - categorical_accuracy: 0.8962 - val_loss: 0.2709 - val_categorical_accuracy: 0.8826
Epoch 3/5
1407/1407 [==============================] - 14s 10ms/step - loss: 0.1942 - categorical_accuracy: 0.9191 - val_loss: 0.2843 - val_categorical_accuracy: 0.8832
Epoch 4/5
1407/1407 [==============================] - 14s 10ms/step - loss: 0.1238 - categorical_accuracy: 0.9482 - val_loss: 0.3423 - val_categorical_accuracy: 0.8848
Epoch 5/5
1407/1407 [==============================] - 14s 10ms/step - loss: 0.0770 - categorical_accuracy: 0.9677 - val_loss: 0.4472 - val_categorical_accuracy: 0.8828


In [20]:
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')